# IAR4 Exercise

**Collision Avoidance:** If the robot detects an obstacle with its front horizontal proximity sensors  (i.e., $\texttt{robot['prox.horizontal'][0] > 1000}, \texttt{robot['prox.horizontal'][1] > 1000}, ...)$, it turns clockwise until the obstacle is not detected anymore. The turning direction is fixed to avoid thrashing. 
 
**Random Walk:** The robot alternates between driving straight forward for $5~\textrm{s}$ and turning in a random direction for~$1~\textrm{s}$. 
  
**Line Following:** If the robot detects the line on the ground with both ground IR sensors, it drives straight forward. 
 If it detects the line only with its right sensor, it turns left. 
 If it detects the line only with its left sensor, it turns right.


In [1]:
import time
from loop_rate_limiters import RateLimiter
import random

# import thymiodirect package
from thymiodirect import Connection 
from thymiodirect import Thymio

In [2]:
# Set to True to use the Thymio simulator
use_simulator = True

# Sets the rate limiter to control the loop frequency in Hz
rate = RateLimiter(frequency=10.0) 

if use_simulator:
    port = 2001
    th = Thymio(use_tcp=True, host='localhost', tcp_port=port, on_connect=lambda node_id: print(f' Thymio {node_id} is connected'))
else:
    port = Connection.serial_default_port()
    th = Thymio(serial_port=port, on_connect=lambda node_id: print(f' Thymio {node_id} is connected'))

print("connecting")
# Connect to Robot
th.connect()
robot = th[th.first_node()]

# Delay to allow robot initialization of all variables
time.sleep(1)

#TODO: 4.1 variables
# Initialize state
state = "find"
ts = time.time()
direction = random.choice(["left", "right"])

# Variable introduced to switch between different behaviors given in the exercise sheet
behavior = "none"

print(th.variables(th.first_node()))


 Thymio 1 is connected


In [ ]:
# don't delete the try-finally block, add your code inside the try block
try:
    # Main loop
    while True:
        proxH = robot['prox.horizontal']
        proxG = robot['prox.ground.reflected']

        # # 2.1: selection mechanism
        # # Activate Collision avoidance behavior
        # Press this button to reset to just collision avoidance and stop random walk behavior
        if robot["button.forward"] == 1:
            state = "find"
            behavior = "none"

        # # Activate Random walk behavior and find line
        if robot["button.left"] == 1:
            behavior = "RW"

        # # Stop the robot
        if robot["button.center"] == 1:
            state = "none"

        #TODO: 4.2 collision avoidance
        if state == "find":
            # Turn clockwise if obstacle detected by front proximity sensors.
            if proxH[0] != 0 or proxH[1] != 0 or proxH[2] != 0 or proxH[3] != 0 or proxH[4] != 0:
                robot['motor.left.target'] = 100
                robot['motor.right.target'] = 0
            else:
                robot['motor.left.target'] = 100
                robot['motor.right.target'] = 100

        #TODO: 4.3 random walk
        if behavior == "RW":
            now = time.time()
            if now - ts >= 5:

                rw_direction = random.choice(["left", "right"])
                if direction == 'left':
                    robot['motor.left.target'] = 0
                    robot['motor.right.target'] = 200
                else:
                    robot['motor.left.target'] = 200
                    robot['motor.right.target'] = 0

                ts = now

            # If line is found, stop the robot.
            if proxG[0] >= 500 and proxG[1] >= 500:
                robot['motor.left.target'] = 0
                robot['motor.right.target'] = 0

        #TODO: 4.4 line following
        # Right now, the robot stops when it finds the line, but it should follow the line

        elif state == "none":
            robot['motor.left.target'] = 0
            robot['motor.right.target'] = 0
        rate.sleep()
finally:
    robot['motor.left.target'] = 0
    robot['motor.right.target'] = 0
    print("Interrupted")

[loop_rate_limiters] [12:31:43] [warning] rate limiter is late by 5049.7 [ms] (rate_limiter.py:96)
